# Aula6 - Doc2Query

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


## Enunciado do Exercício

Fazer o finetuning de um buscador denso

Usar como treino o dataset "tiny" do MS MARCO
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

Avaliar o modelo no TREC-COVID, e comparar os resultados com o BM25 e doc2query

Comparar busca "exaustiva" (semelhança do vetor query com todos os vetores do corpus) com a busca aproximada (Approximate Nearest Neighbor - ANN)

Para a busca aproximada, usar os algoritmos existentes na biblioteca sentence-transformers (ex: hnswlib) OU implemente um você mesmo (Bonus!)

Dicas:

    Usar a média dos vetores da última camada (conhecido como mean pooling) do transformer para representar queries e passagens; Alternativamente, usar apenas o vetor do [CLS] da última cada.
    Tente inicialmente uma loss fácil de implementar, como a entropia-cruzada
    Começar o treino a partir do microsoft/MiniLM-L12-H384-uncased
    Avaliar o pipeline usando um modelo já bem treinado: sentence-transformers/all-mpnet-base-v2
    Comparar resultados usando semelhança de cosseno e produto escalar como funções de similaridade
    Para checar se seu codigo de avaliação está correto, comparar o seu desempenho com o do modelo já treinado no MS MARCO:   https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2; O nDCG@10 no TREC-COVID deve ser ~0.47
    Usar a biblioteca do sentence-transformers para avaliar o modelo


## Fase

Utilizando DPR treinado

# Organizando o ambiente

In [1]:
import pickle

In [2]:
import os

In [3]:
import gzip

In [4]:
import json

In [5]:
import pandas as pd

In [6]:
import faiss

In [49]:
from tqdm import tqdm

In [40]:
from transformers import AutoTokenizer, AutoModel
import torch

In [133]:
from evaluate import load

In [7]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr'

In [8]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


In [9]:
DIRETORIO_RUN = f"{DIRETORIO_TRABALHO}/runs"
CAMINHO_RUN = f"{DIRETORIO_RUN}/run-trec-covid-bm25.txt"

In [10]:
if os.path.exists(DIRETORIO_RUN):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_RUN)
    print('pasta criada!')


pasta já existia!


In [11]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [12]:
from pyserini.search.lucene import LuceneSearcher

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  

In [ ]:
from psutil import virtual_memory

In [ ]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 54.49
 used 11.28
 free 36.91
 cached 17.01
 buffers 2.16
/nGPU
Sun Apr 16 12:03:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 52%   50C    P8    40W / 370W |     64MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------

## Fixando as seeds

In [14]:
import random
import torch
import numpy as np

In [15]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [16]:
num_semente=123
inicializa_seed(num_semente)

## Preparando para debug e display

In [17]:
import pandas as pd

In [18]:
#!pip install transformers -q

In [19]:
import transformers

In [20]:
# https://zohaib.me/debugging-in-google-collab-notebook/
# !pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [21]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [22]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [23]:
config_display()

In [24]:
config_debug()

# Baixando o dataset para avaliação (trec-covid)

## Queries

In [25]:
from pyserini.search import get_topics

In [26]:
topics = get_topics('covid-round5')
print(f'{len(topics)} queries total')

50 queries total


In [42]:
topics[50]

{'question': 'what is known about an mRNA vaccine for the SARS-CoV-2 virus?',
 'query': 'mRNA vaccine coronavirus',
 'narrative': 'Looking for studies specifically focusing on mRNA vaccines for COVID-19, including how mRNA vaccines work, why they are promising, and any results from actual clinical studies.'}

## Relevância (qrel) de teste

In [43]:
if not os.path.exists(f'{DIRETORIO_TRABALHO}/test.tsv'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
    !mv test.tsv {DIRETORIO_TRABALHO}/

In [44]:
qrel = pd.read_csv(f"{DIRETORIO_TRABALHO}/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])

In [45]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [47]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [48]:
qrel["q0"] = "q0"
qrel_dict = qrel.to_dict(orient="list")

In [34]:
qrel_dict['query'][0], qrel_dict['docid'][0], qrel_dict['rel'][0]

(1, '005b2j4b', 2)

## Documentos a serem indexados

In [35]:
if not os.path.exists(f"{DIRETORIO_TRABALHO}/corpus.jsonl.gz"):
    !wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
    !mv corpus.jsonl.gz {DIRETORIO_TRABALHO}

In [36]:
# Descompacte o arquivo para a memória
with gzip.open(f'{DIRETORIO_TRABALHO}/corpus.jsonl.gz', 'rt') as f:
    # Leia o conteúdo do arquivo descompactado
    corpus = [json.loads(line) for line in f]

In [50]:
# Exiba os dados carregados
print(f"{type(corpus)} len(corpus): {len(corpus)} corpus[0] {corpus[0]}" )

<class 'list'> len(corpus): 171332 corpus[0] {'_id': 'ug7v899j', 'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%

In [51]:
corpus[0].keys()

dict_keys(['_id', 'title', 'text', 'metadata'])

# Busca para modelo já treinado all-MiniLM-L12-v2

## Geração dos embeddings 

Para checar se seu codigo de avaliação está correto, comparar o seu desempenho com o do modelo já treinado no MS MARCO:   https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2; O nDCG@10 no TREC-COVID deve ser ~0.47

In [52]:
MODEL_NAME = 'sentence-transformers/all-MiniLM-L12-v2'

In [53]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
passages_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
topics_model = AutoModel.from_pretrained(MODEL_NAME).to(device)

In [54]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [78]:
def gera_embeddings(parm_model:AutoModel , parm_tokenizer:AutoTokenizer, parm_sentences:list, parm_tipo_resumo:str='cls'):
    """
    Função para gerar embeddings de sentenças usando um modelo pré-treinado.

    Args:
        parm_model (AutoModel): Modelo pré-treinado para geração de embeddings.
        parm_tokenizer (AutoTokenizer): Tokenizer associado ao modelo pré-treinado.
        parm_sentences (list): Lista de sentenças para as quais os embeddings serão gerados.
        parm_tipo_resumo (str, opcional): Tipo de resumo a ser aplicado nas sentenças. Pode ser 'cls' para usar o token [CLS]
            ou 'mean' para usar a média das embeddings dos tokens. O padrão é 'cls'.

    Returns:
        embeddings (torch.Tensor): Embeddings gerados para as sentenças, como um tensor do PyTorch.
    """

    # Tokenize sentences
    encoded_input = parm_tokenizer(parm_sentences, padding=True, truncation=True, return_tensors='pt')
    # Executa o tokenizador nas sentenças fornecidas, com opções de padding, truncation e retorno como tensores do PyTorch

    # Move os dados para o dispositivo especificado (CPU ou GPU)
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}

    with torch.no_grad():
        # Desativa o cálculo de gradientes para economizar memória e acelerar a inferência
        model_output = parm_model(**encoded_input)
        # Passa os inputs tokenizados para o modelo e obtém a saída do modelo

    if parm_tipo_resumo == 'cls':
        # Se o tipo de resumo for 'cls', retorna o embedding do token [CLS]
        embeddings = model_output[0][:, 0, :].to('cpu').numpy()
        # Seleciona o embedding do token [CLS], que está na primeira posição do output do modelo
    elif parm_tipo_resumo == 'mean':
        # Se o tipo de resumo for 'mean', realiza pooling médio nas embeddings dos tokens
        embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        # Chama a função mean_pooling para realizar o pooling médio, passando o output do modelo e a máscara de atenção
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1).to('cpu').numpy()
        # Normaliza as embeddings usando a norma L2 (euclidiana)
    else:
        # Se o tipo de resumo não for válido, levanta uma exceção
        raise Exception(f"parm_tipo_resumo deve ser cls ou mean, não  {parm_tipo_resumo}")

    return embeddings
    # Retorna as embeddings geradas como um tensor do PyTorch


### Experimentos de teste de encoding

In [56]:
sentences_test = ["This is an example sentence", "Each sentence is converted"]

In [57]:
# Tokenize sentences
encoded_input = tokenizer(sentences_test, padding=True, truncation=True, return_tensors='pt')

In [58]:
# Move os dados para o dispositivo especificado (CPU ou GPU)
encoded_input = {key: value.to(device) for key, value in encoded_input.items()}

In [59]:
encoded_input

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
         [ 101, 2169, 6251, 2003, 4991,  102,    0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 0]], device='cuda:0')}

In [60]:
# Compute token embeddings
with torch.no_grad():
    model_output = passages_model(**encoded_input)

In [61]:
cls_embeddings = model_output[0][:, 0, :]


In [62]:
cls_embeddings.shape

torch.Size([2, 384])

In [63]:
# Perform pooling
mean_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [64]:
# Normalize embeddings
mean_embeddings = torch.nn.functional.normalize(mean_embeddings, p=2, dim=1)


In [65]:

print("Sentence embeddings:")
print(mean_embeddings.shape,mean_embeddings[0][:10], '\n(...)\n', mean_embeddings[0][-10:])


Sentence embeddings:
torch.Size([2, 384]) tensor([-2.0266e-04, 8.1480e-02, 3.1362e-02, 2.9206e-03, 2.6156e-02, 2.9074e-02, 7.8262e-02, -1.8042e-03,
        1.0134e-01, -4.5171e-02], device='cuda:0') 
(...)
 tensor([3.3426e-02, -3.7150e-02, 9.1379e-02, 4.6740e-03, -1.4017e-02, 2.1975e-02, 2.3848e-02, 6.2093e-02,
        4.9103e-02, -2.9018e-04], device='cuda:0')


In [66]:
model_output[-1].shape

torch.Size([2, 384])

In [67]:
model_output[0].size()

torch.Size([2, 7, 384])

In [70]:
len(model_output)

2

In [71]:
model_output[1].shape

torch.Size([2, 384])

## Gerando enconding para queries

In [79]:
%%time
topics_encoded = {}
for id, value in topics.items():
    # print(id, value)
    topics_encoded[id]= gera_embeddings(passages_model, tokenizer, [value['question']], 'cls')[0]

CPU times: user 503 ms, sys: 16 ms, total: 519 ms
Wall time: 518 ms


In [80]:
len(topics_encoded), topics_encoded[1][:10]

(50,
 array([-0.16807137, -0.04519013, -0.21228197, -0.14981315,  0.37164712,
        -0.10712396,  0.03028835,  0.15896398,  0.19999021, -0.30677956],
       dtype=float32))

## Gerando enconding para docs

In [81]:
corpus[0].keys()

dict_keys(['_id', 'title', 'text', 'metadata'])

In [82]:
%%time
corpus_encoded = {}
for docto in tqdm(corpus):
    corpus_encoded[docto['_id']] = gera_embeddings(passages_model, tokenizer, [docto['title'] + '. '+ docto['text']], 'cls')[0]
    # print(docto)
    # break

100%|██████████| 171332/171332 [33:19<00:00, 85.67it/s]

CPU times: user 33min 22s, sys: 1.79 s, total: 33min 24s
Wall time: 33min 19s


In [83]:
len(corpus_encoded)

171332

In [84]:
corpus_encoded['ug7v899j']

array([-6.21874258e-03, -1.20662630e-01, -2.61611998e-01, -1.97607756e-01,
        8.76287147e-02, -3.27481814e-02, -2.19035655e-01,  1.07552670e-01,
       -4.28966321e-02,  9.19836536e-02,  5.32585103e-03,  1.73973653e-03,
        2.96231002e-01, -2.79507518e-01,  1.46820173e-01, -8.19274411e-02,
       -8.32051970e-03,  5.73352911e-03,  2.05598041e-01, -2.91884132e-02,
        7.45842755e-02,  1.09343365e-01,  1.60747126e-01, -1.14155456e-01,
        4.72229309e-02, -1.31777212e-01, -6.65777624e-02,  1.38052665e-02,
        4.57048774e-01, -9.10129189e-01, -1.49422109e-01, -4.31542397e-02,
        2.46372789e-01, -1.72781363e-01, -6.20418368e-03,  3.72753531e-01,
        1.91106677e-01,  3.71161520e-01,  4.24360968e-02, -7.49465525e-02,
        5.43528572e-02,  1.71901554e-01,  5.26305847e-02, -3.62779170e-01,
        2.85411673e-03, -2.45256782e-01,  1.46584049e-01, -1.99391931e-01,
       -1.75008059e-01,  1.29311651e-01,  5.74190393e-02,  1.31374868e-02,
       -4.77273166e-02,  

### Salvando os dados

In [85]:
with open(f"{DIRETORIO_TRABALHO}/data_index_all_minilm.pickle", 'wb') as outputFile:
    pickle.dump({'topics_encoded': topics_encoded,
                 'corpus_encoded': corpus_encoded}, outputFile, pickle.HIGHEST_PROTOCOL)

In [96]:
len(corpus_encoded)

171332

## Indexação 

## Lendo os dados

In [ ]:
with open(f"{DIRETORIO_TRABALHO}/data_index_all_minilm.pickle", "rb") as f:
  teste = pickle.load(f)

In [ ]:
topics_encoded = teste['topics_encoded']
corpus_encoded = teste['corpus_encoded']

In [97]:
assert len(corpus_encoded)==171332, f"Tamanho len(corpus_encoded) deveria ser 171332 e foi {len(corpus_encoded)}"
assert len(topics_encoded)==50, f"Tamanho len(topics_encoded) deveria ser 171332 e foi {len(topics_encoded)}"

In [1]:
def gera_indice(parm_corpus_encoded:{}, parm_tipo_similaridade_indice:str='cosseno'):
    """
        Ainda não implementados outros tipos de similaridade.
        Outras métricas de similaridade suportadas pelo FAISS incluem a distância euclidiana (faiss.IndexFlatL2) e 
        a distância Manhattan (faiss.IndexFlatL1).
        Você pode consultar a documentação oficial do FAISS para obter mais informações sobre as métricas disponíveis e como usá-las: https://github.com/facebookresearch/faiss/blob/master/docs/indexes.md#available-indexes
    """
    lista_id_docto_corpus = list(parm_corpus_encoded.keys())
    corpus_encoded_np = np.array(list(parm_corpus_encoded.values()))
    embed_dim = parm_corpus_encoded[lista_id_docto_corpus[0]].shape[0]  # dimensão dos embeddings
    print(f"embed_dim: {embed_dim}")    
    if parm_tipo_similaridade_indice == 'cosseno':
        #  a métrica cosseno é usada com o índice FlatIP, que é uma versão otimizada para a métrica cosseno do índice Flat, que é o índice padrão do FAISS.
        index = faiss.IndexFlatL2(embed_dim)  # índice com métrica L2 (euclidiana)
    else:
        raise Exception(f"parm_tipo_similaridade_indice deve ser cosseno, não  {parm_tipo_similaridade_indice}")
    index.add(corpus_encoded_np)  # adiciona os embeddings do corpus ao índice
    return index, lista_id_docto_corpus

In [99]:
indice, lista_id_docto_corpus = gera_indice(corpus_encoded, 'cosseno')


embed_dim: 384


In [100]:
type(indice)

faiss.swigfaiss_avx2.IndexFlatL2

In [101]:
lista_id_docto_corpus[:4]

['ug7v899j', '02tnwd4m', 'ejv2xln0', '2b73a28n']

## Realizar buscas 

In [129]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries_embed_index(parm_topics_encoded:{}, parm_indice_com_embeddings:faiss.swigfaiss_avx2.IndexFlatL2, parm_lista_id_docto_corpus:list, parm_num_max_hits:int=1000):
    """
    A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
    Usada no notebook da aula 2

    Parâmetros:

    file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
    topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
    searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
    num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
    Retorno:

    A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
    Comentário:

    A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
    O número de consultas concluídas é impresso a cada 100 consultas.
    """
    topics_encoded_np = np.array(list(parm_topics_encoded.values()))

    scores_result_search, index_result_search = parm_indice_com_embeddings.search(topics_encoded_np, parm_num_max_hits)  # realiza a pesquisa no índice

    print(f"Após pesquisa, scores_result_search.shape: {scores_result_search.shape}, scores_result_search[0][:10]:{scores_result_search[0][:10]} ")


    # ajustando os ids

    all_topics_results = {}
    for ndx_query, (topic_id, doc_indices) in enumerate(zip(topics_encoded.keys(), index_result_search)):
        doc_scores_query = scores_result_search[ndx_query]
        topic_results = []
        for ndx_doc, doc_ndx in enumerate(doc_indices):
            doc_id = parm_lista_id_docto_corpus[doc_ndx]
            doc_score = doc_scores_query[ndx_doc]
            topic_results.append([doc_id, doc_score])
        topic_results.sort(key=lambda x: x[1], reverse=True)
        all_topics_results[topic_id] = topic_results
    print(len(all_topics_results), list(all_topics_results.keys()))# , list(all_topics_results.values())[0][:10])
    print(f"Exemplo de um resultado: query_id== {list(all_topics_results.keys())[0]}, first_10_relevant_documents_retrieved = {list(all_topics_results.values())[0][:10]}")




    with open(CAMINHO_RUN, 'w') as runfile:
        for topic_id in topics_encoded:
            # print(topic_id)
            for i in range(0, parm_num_max_hits):
                _ = runfile.write(f'{topic_id} Q0 {all_topics_results[topic_id][i][0]} {i+1} {all_topics_results[topic_id][i][1]:.6f} Pesquisa\n')
                


In [130]:
run_all_queries_embed_index(topics_encoded, indice, lista_id_docto_corpus, parm_num_max_hits=1000)

Após pesquisa, scores_result_search.shape: (50, 1000), scores_result_search[0][:10]:[3.0686874 3.1987457 3.482605  3.7657661 3.9926224 4.0703087 4.1311874
 4.42017   4.4837875 4.828064 ] 
50 [44, 45, 46, 47, 48, 49, 50, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
Exemplo de um resultado: query_id== 44, first_10_relevant_documents_retrieved = [['rroxtmag', 13.564629], ['6qs8pvrv', 13.563751], ['wiomb5k3', 13.5636635], ['id3iu30t', 13.562904], ['5ina237p', 13.562317], ['5r3kf9wa', 13.560501], ['xfjexm5b', 13.560413], ['xb4klcub', 13.557503], ['h9hipko0', 13.552311], ['k2pha0ub', 13.551182]]


## Avaliação dos resultados

In [134]:
trec_eval = load("trec_eval")

In [135]:
### Calculando métricas
run = pd.read_csv(f"{CAMINHO_RUN}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


   query  q0     docid  rank      score    system
0     44  Q0  rroxtmag     1  13.564629  Pesquisa
1     44  Q0  6qs8pvrv     2  13.563751  Pesquisa
2     44  Q0  wiomb5k3     3  13.563663  Pesquisa
3     44  Q0  id3iu30t     4  13.562904  Pesquisa
4     44  Q0  5ina237p     5  13.562317  Pesquisa

[5 rows x 6 columns]
NDCG@10: 0.02728446458793006
Resultados: {'runid': 'Pesquisa', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 4111, 'num_q': 50, 'map': 0.020725983829006842, 'gm_map': 0.0032868832686071886, 'bpref': 0.16421888309534055, 'Rprec': 0.05620782552194098, 'recip_rank': 0.0958358681934901, 'P@5': 0.04, 'P@10': 0.04, 'P@15': 0.04, 'P@20': 0.040999999999999995, 'P@30': 0.04333333333333333, 'P@100': 0.0426, 'P@200': 0.04540000000000001, 'P@500': 0.05316, 'P@1000': 0.08222, 'NDCG@5': 0.024541013967736877, 'NDCG@10': 0.02728446458793006, 'NDCG@15': 0.028399203726244823, 'NDCG@20': 0.03007681061382414, 'NDCG@30': 0.03231653413695874, 'NDCG@100': 0.03423262008119381, 'NDCG@200':